# Library Import

In [ ]:
import pandas as pd
import numpy as np
from pathlib import Path

import analyzer
import neural_network as nn
import convolutional_neural_network as cnn
import visualizer

# Training Data Set Preparation

In [ ]:
resolution = [250,250]
# Resolution should be consistent throughout the file

In [ ]:

# Because the training dataset are individual, the list of each data just contain themselves.
data1 = analyzer.Data("MORE_DATA/db_Y_0027.okc", resolution) 
data2 = analyzer.Data("MORE_DATA/db_Y_0030.okc", resolution)


# Training The Model

In [ ]:
# Run this section if you want to use NN model

array1, headers1, non_nan_indices1, num_grids1 = nn.data_arranger(data1.df)
array2, headers2, non_nan_indices2, num_grids2 = nn.data_arranger(data2.df)

# The learning rate, batch size, and epochs are proven to be working.

nn_model = nn.model_create_compile(headers1, 0.05)

nn_model, loss_hist = nn.model_train(nn_model, array1, 1000, 10)
nn_model, loss_hist = nn.model_train(nn_model, array2, 1000, 10)

In [ ]:
# Run this section if you want to use CNN model

array1, headers1, indices1 = cnn.data_arranger(data1.df, resolution)
array2, headers2, indices2 = cnn.data_arranger(data2.df, resolution)

# The learning rate and epochs are proven to be working.

cnn_model = cnn.model_2D_create_compile(headers1, 0.05, resolution)

cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array1, 3)
cnn_model, loss_hist = cnn.model_2D_train(cnn_model, array2, 3)

# Data Classification

Choose codes from these 3 codes below to run if you want to classify few individual data.

In [ ]:
data3 = analyzer.Data("MORE_DATA/db_Y_0049.okc", resolution)

In [ ]:
# Run this code if your model choice is NN
array3, headers3, non_nan_indices3, num_grids3 = nn.data_arranger(data3.df)
data3.df = nn.model_classification(nn_model, array3, non_nan_indices3, num_grids3, data3.df, False)

In [ ]:
# Run this code if your model choice is CNN
array3, headers3, indices3 = cnn.data_arranger(data3.df, resolution)
data3.df = cnn.model_2D_classification(cnn_model, array3, indices3, data3.df, False)

Choose codes from these 3 codes below to run if you want to classify a series of data.

In [ ]:
list_paths_classify = [f"MORE_DATA/db_Y_{i:04d}.okc" for i in range(99)]

Data_classify = [analyzer.Data(path, list_paths_classify, resolution) for path in list_paths_classify] 

In [ ]:
# Run this code if your model choice is NN
for data in Data_classify:
    array, headers, non_nan_indices, num_grids = nn.data_arranger(data.df)
    data.df = nn.model_classification(nn_model, array, non_nan_indices, num_grids, data.df, False)

In [ ]:
# Run this code if your model choice is CNN
for data in Data_classify:
    array, headers, indices = cnn.data_arranger(data.df, data.resolution)
    data.df = cnn.model_2D_classification(cnn_model, array, indices, data.df, False)

# Data Export

Choose codes from these 2 codes below to run if you want to classify few individual data.

In [ ]:
# Run this code to export the image
visualizer.plot_2D_df(data3.df, 'is_boundary', 'classification.png')

In [ ]:
# Run this code to export the csv file
data3.df.to_csv('classification.csv', index=False)

Choose codes from these 2 codes below to run if you want to classify a series of data.

In [ ]:
# Run this code to export the image
for i, data in enumerate(Data_classify):
    visualizer.plot_2D_df(data.df, 'is_boundary', f'classification_{i}.png')

In [ ]:
# Run this code to export the csv file
for i, data in enumerate(Data_classify):
    data.df.to_csv(f'classification_{i}.csv', index=False)

# Temporary Code Zone
---

### Merge CSV

In [ ]:
import os
import csv

path = '3D_DATA/small_db_result'
output_file = '3D_DATA/small_db.csv'

files = [file for file in os.listdir(path) if file.endswith('.csv')]

with open(output_file, 'w', newline='', encoding='utf-8') as outfile:
    writer = None
    
    for index, filename in enumerate(files):
        print(f"merging file: {index + 1}/{len(files)}", flush=True)
        
        with open(os.path.join(path, filename), 'r', encoding='utf-8') as infile:
            reader = csv.reader(infile)
            
            header = next(reader)
            
            if writer is None:
                writer = csv.writer(outfile)
                writer.writerow(header)
            
            for row in reader:
                writer.writerow(row)

print("All files merged successfully!")


### Plot Streamline 2D

In [ ]:
from visualizer import plot_3D_to_2D_slice_streamline

plot_3D_to_2D_slice_streamline(input_file="3D_DATA/ra10e7_result/ra10e7_100.csv", output_file="streamlines.html", direction='y', seed_points_resolution=[20,20], max_time=0.2, cmap = 'viridis', axis_limits=[-0.5,0.5,0,1])

### Plot Streamline 3D

In [1]:
import pyvista as pv
import scipy
import numpy as np
import vtk
import pandas as pd

def arrange_slice_df(input_file: str, direction:str, output_df: str = None, cubic: bool = True) -> pd.DataFrame:
    '''
    Arrange sliced data to prepare for plotting streamlines.
    Fill empty rows with coordinates to ensure a complete grid of points in a square.
    For NaN points outside the container, set velocity to 0.
    For NaN points inside the container, interpolate velocity as needed.

    Note that because even data for 3D are stored in many slices, so it is necessary to specify a slicing direction.
    Args:
        input_file: The path of the SLICED CSV file (preprocessed by our PlumeCNN).
        Direction: Specify the axis to which the slice is perpendicular ('x', 'y', or 'z'). 
        output_file: If you want to output the arranged data to CSV, please input the desired path of the output file.
            If None(in default), it will not write any file out.
        if_cubic: If True (default), the function will process coordinates and velocity components in all directions.
            If false, Only coordinates and velocity components in the remaining directions will be considered. 
            The reason why this variable is called 'cubic' is because only a slice representing for a whole cubic data needs this option.
           
    Returns:The arranged pandas dataframe, containing coordinate values and velocities in desired directions.
    '''
    df = pd.read_csv(input_file)
    # Read and extract columns to read
    if direction == 'z':
        coord1 = 'x'
        coord2 = 'y'
        coord3 = 'z'
        vel1 = 'x_velocity'
        vel2 = 'y_velocity'
        vel3 = 'z_velocity'
    elif direction == 'y':
        coord1 = 'x'
        coord2 = 'z'
        coord3 = 'y'
        vel1 = 'x_velocity'
        vel2 = 'z_velocity'
        vel3 = 'y_velocity'
    elif direction == 'x':
        coord1 = 'y'
        coord2 = 'z'
        coord3 = 'x'
        vel1 = 'y_velocity'
        vel2 = 'z_velocity'
        vel3 = 'x_velocity'

    if cubic is True:
        df = df[['x', 'y', 'z', 'x_velocity', 'y_velocity', 'z_velocity']].dropna()
    else:
        df = df[[coord1, coord2, vel1, vel2]].dropna()

    # Extract unique coordinate values, sorted in ascending order
    coord1_values = np.sort(df[coord1].unique())
    coord2_values = np.sort(df[coord2].unique())
    # define grid points in the specified plane for interpolation
    grid_coord1, grid_coord2 = np.meshgrid(coord1_values, coord2_values, indexing='ij')
    # Prepare data points and velocity components for interpolation
    points = df[[coord1, coord2]].values
    u_values = df[vel1].values
    v_values = df[vel2].values
    if cubic is True:
        w_values = df[vel3].values
    # Interpolate velocity components onto the grid, to eliminate NaN points lying in the dataset
    '''without this step, NaN points in the grid will make the streamline extremely short'''
    u_grid = scipy.interpolate.griddata(points, u_values, (grid_coord1, grid_coord2), method='linear')
    v_grid = scipy.interpolate.griddata(points, v_values, (grid_coord1, grid_coord2), method='linear')
    if cubic is True:
        w_grid = scipy.interpolate.griddata(points, w_values, (grid_coord1, grid_coord2), method='linear')
    # Replace any NaN values in interpolated data with zeros, to specify the border of the valid data
    u_grid = np.nan_to_num(u_grid, nan=0.0).ravel(order='F')
    v_grid = np.nan_to_num(v_grid, nan=0.0).ravel(order='F')
    if cubic is True:
        w_grid = np.nan_to_num(w_grid, nan=0.0).ravel(order='F')
    
    # Prepare the final DataFrame for return
    if direction == 'z':
        result_df = pd.DataFrame({
            'x': grid_coord1.ravel(order='F'),
            'y': grid_coord2.ravel(order='F'),
            'x_velocity': u_grid,
            'y_velocity': v_grid
        })
        if cubic is True:
            result_df['z'] = df[coord3].unique().mean()
            result_df['z_velocity'] = w_grid
    elif direction == 'y':
        result_df = pd.DataFrame({
            'x': grid_coord1.ravel(order='F'),
            'z': grid_coord2.ravel(order='F'),
            'x_velocity': u_grid,
            'z_velocity': v_grid
        })
        if cubic is True:
            result_df['y'] = df[coord3].unique().mean()
            result_df['y_velocity'] = w_grid
    elif direction == 'x':
        result_df = pd.DataFrame({
            'y': grid_coord1.ravel(order='F'),
            'z': grid_coord2.ravel(order='F'),
            'y_velocity': u_grid,
            'z_velocity': v_grid
        })
        if cubic is True:
            result_df['x'] = df[coord3].unique().mean()
            result_df['x_velocity'] = w_grid
    # Save to CSV if output path is provided
    if output_df is not None:
        result_df.to_csv(output_df, index=False)

    # Return the result DataFrame
    return result_df

def plot_3D_to_2D_slice_streamline(input_file: str, output_file: str, direction: str, seed_points_resolution: list, integration_direction: str = 'both', max_time: float = 0.2, terminal_speed: float = 1e-5, cmap: str = 'viridis', axis_limits: list = None):
    '''
    Generate and visualize 2D streamlines from a 3D dataset, projected onto a specified plane,
    and export the visualization as an HTML file. The seed points (starting points) of the streamlines are evenly distributed, with the resolution specified by the user.

    Args:
        input_file: Path to the CSV file containing the 3D dataset.
        output_file: Path to the output HTML file for the visualization, sontaining the extension.
        direction: The direction to which the plane is perpendicular ('x', 'y' or 'z').
        seed_points_resolution: Specifies the resolution for distributing seed points in the format [var1_resolution, var2_resolution], where each element controls the density along the respective variable axis.
        integration_direction (optional): Specify whether the streamline is integrated in the upstream or downstream directions (or both). Options are 'both'(default), 'backward', or 'forward'.
        max_time (optional): What is the maximum integration time of a streamline (0.2 in default).
        terminal_speed (optional): When will the integration stop (1e-5 in default).
        cmap (optional): Colormap to use for the visualization. Default is 'viridis'.
        axis_limits (optional): A list of axis limits for the plot in the form [var1_min, var1_max, var2_min, var2_max].
            If None, the axis limits will be determined automatically from the data.
    '''

    # Suppress all VTK warnings and errors
    '''
    If not, we will get the warning "Unable to factor linear system" for every streamline we plot, although the result is quite good.
    '''
    vtk.vtkObject.GlobalWarningDisplayOff()

    # Define coordinate and velocity components based on the specified direction
    if direction == 'z':
        coord1 = 'x'
        coord2 = 'y'
        vel1 = 'x_velocity'
        vel2 = 'y_velocity'
    elif direction == 'y':
        coord1 = 'x'
        coord2 = 'z'
        vel1 = 'x_velocity'
        vel2 = 'z_velocity'
    elif direction == 'x':
        coord1 = 'y'
        coord2 = 'z'
        vel1 = 'y_velocity'
        vel2 = 'z_velocity'
    else:
        raise ValueError("Invalid direction. Choose from 'x', 'y', 'z'.")
    
    # Load and prepare data
    df = arrange_slice_df(input_file, direction, cubic=False)

    # Extract unique coordinate values, sorted in ascending order
    coord1_values = np.sort(df[coord1].unique())
    coord2_values = np.sort(df[coord2].unique())

    # Calculate the 'dx' between coordinate values for grid step size
    d_coord1 = np.diff(coord1_values).mean()
    d_coord2 = np.diff(coord2_values).mean()

    grid_coord1, grid_coord2 = np.meshgrid(coord1_values, coord2_values, indexing='ij')

    # Step 2: Create a structured 3D grid for visualization
    # Initialize x, y, z coordinates for the structured grid
    if direction == 'z':
        grid = pv.StructuredGrid(grid_coord1, grid_coord2, np.zeros_like(grid_coord1))
    elif direction == 'y':
        grid = pv.StructuredGrid(grid_coord1, np.zeros_like(grid_coord1), grid_coord2)
    elif direction == 'x':
        grid = pv.StructuredGrid(np.zeros_like(grid_coord1), grid_coord1, grid_coord2)

    # Combine velocity components into a single array and assign to the grid
    if direction == 'z':
        velocity = np.column_stack((df[vel1], df[vel2], np.zeros_like(df[vel1])))
    elif direction == 'y':
        velocity = np.column_stack((df[vel1], np.zeros_like(df[vel1]), df[vel2]))
    elif direction == 'x':
        velocity = np.column_stack((np.zeros_like(df[vel1]), df[vel1], df[vel2]))

    grid.point_data['velocity'] = velocity

    # Step 3: Generate streamlines from seed points (initial points)
    # Define seed points across the flow domain
    seed_coord1, seed_coord2 = np.meshgrid(
        np.linspace(coord1_values[0], coord1_values[-1], seed_points_resolution[0]),
        np.linspace(coord2_values[0], coord2_values[-1], seed_points_resolution[1])
    )
    seed_coord1 = seed_coord1.ravel()
    seed_coord2 = seed_coord2.ravel()

    if direction == 'z':
        x_seed = seed_coord1
        y_seed = seed_coord2
        z_seed = np.zeros_like(x_seed)
    elif direction == 'y':
        x_seed = seed_coord1
        y_seed = np.zeros_like(seed_coord1)
        z_seed = seed_coord2
    elif direction == 'x':
        x_seed = np.zeros_like(seed_coord1)
        y_seed = seed_coord1
        z_seed = seed_coord2

    # Combine coordinates to form seed points
    seed_points = np.column_stack((x_seed, y_seed, z_seed))
    seed = pv.PolyData(seed_points)

    # Calculate streamlines based on velocity field, starting from the seed points
    streamlines = grid.streamlines_from_source(
        source=seed,
        vectors='velocity',
        integration_direction=integration_direction,
        max_time=max_time,
        initial_step_length=0.5*(d_coord1+d_coord2),
        terminal_speed=terminal_speed
    )

    # Calculate and add velocity magnitude as a scalar field on streamlines for coloring
    velocity_vectors = streamlines['velocity']
    velocity_magnitude = np.linalg.norm(velocity_vectors, axis=1)
    streamlines['velocity_magnitude'] = velocity_magnitude

    # Step 4: Visualize and export streamlines as HTML
    plotter = pv.Plotter(off_screen=True)
    plotter.add_mesh(grid.outline(), color='k')

    # Color the streamlines by velocity magnitude and set up a scalar bar
    plotter.add_mesh(
        streamlines.tube(radius=0.5 * (d_coord1+d_coord2) * 0.5),
        scalars='velocity_magnitude',
        cmap=cmap,  # Use the colormap specified in the function argument
        scalar_bar_args={'title': 'Velocity Magnitude'}
    )
    if direction == 'z':
        # Set the view based on the specified direction
        plotter.view_xy()
    elif direction == 'y':
        plotter.view_xz()
    elif direction == 'x':
        plotter.view_yz()

    # Show grid with axis labels
    plotter.show_grid(
        xtitle='X',
        ytitle='Y',
        ztitle='Z',
        grid='front'  # Display the grid in front of the scene
    )

    # Adjust axis limits by adding an invisible box
    if axis_limits is not None:
        var1_min, var1_max, var2_min, var2_max = axis_limits

        if direction == 'z':
            # For 'z' direction, create a box with x and y limits
            box = pv.Box(bounds=(
                var1_min, var1_max,  # x bounds
                var2_min, var2_max,  # y bounds
                0, 0                # z bounds (since it's a 2D plane at z=0)
            ))
        elif direction == 'y':
            # For 'y' direction, create a box with x and z limits
            box = pv.Box(bounds=(
                var1_min, var1_max,  # x bounds
                0, 0,               # y bounds (since it's a 2D plane at y=0)
                var2_min, var2_max   # z bounds
            ))
        elif direction == 'x':
            # For 'x' direction, create a box with y and z limits
            box = pv.Box(bounds=(
                0, 0,               # x bounds (since it's a 2D plane at x=0)
                var1_min, var1_max,  # y bounds
                var2_min, var2_max   # z bounds
            ))

        # Add the box to the plotter with zero opacity
        plotter.add_mesh(box, opacity=0.0, show_edges=False)

    plotter.export_html(output_file)

plot_3D_to_2D_slice_streamline("3D_DATA/ra10e7_result/ra10e7_100.csv", "streamline.html", 'y', [20,20])